# Importer les bibliothéques

In [200]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Importer dataset

In [201]:
dt=pd.read_csv("./dataCleaned.csv")

In [202]:
dt.head()

,Unnamed: 0,team_code,Opponent_code,season,Round_code,Venue_code,Referee_code,Formation_code,Day_code,GF_rolling,GA_rolling,win_rolling,draw_rolling,lose_rolling,Result_code
0,3,0,5,2010,32,1,41,19,5,3.000000,0.666667,0.666667,0.333333,0.000000,2
1,4,0,31,2010,33,0,35,12,5,4.000000,0.666667,1.000000,0.000000,0.000000,0
2,5,0,35,2010,34,1,25,12,5,2.333333,1.000000,0.666667,0.333333,0.000000,1
3,6,0,11,2010,35,0,27,19,6,2.333333,1.666667,0.333333,0.333333,0.333333,1
4,7,0,2,2010,36,1,24,19,5,1.000000,2.000000,0.000000,0.333333,0.666667,2


In [203]:
train=dt[dt["season"]<2020]

In [204]:
test=dt[dt["season"]>=2020]

In [205]:
x_train=train.drop(["Result_code"],axis=1)
y_train=train["Result_code"]
x_test=test.drop(["Result_code"],axis=1)
y_test=test["Result_code"]


In [206]:
x_train=x_train.drop(["Unnamed: 0"],axis=1)
x_test=x_test.drop(["Unnamed: 0"],axis=1)

In [207]:
x_train

,team_code,Opponent_code,season,Round_code,Venue_code,Referee_code,Formation_code,Day_code,GF_rolling,GA_rolling,win_rolling,draw_rolling,lose_rolling
0,0,5,2010,32,1,41,19,5,3.000000,0.666667,0.666667,0.333333,0.000000
1,0,31,2010,33,0,35,12,5,4.000000,0.666667,1.000000,0.000000,0.000000
2,0,35,2010,34,1,25,12,5,2.333333,1.000000,0.666667,0.333333,0.000000
3,0,11,2010,35,0,27,19,6,2.333333,1.666667,0.333333,0.333333,0.333333
4,0,2,2010,36,1,24,19,5,1.000000,2.000000,0.000000,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,38,28,2019,27,0,24,4,2,0.666667,0.666667,0.666667,0.000000,0.333333
9645,38,13,2019,28,1,2,4,6,0.333333,1.000000,0.333333,0.000000,0.666667
9646,38,9,2019,29,0,27,4,2,1.000000,1.000000,0.333333,0.000000,0.666667
9647,38,12,2019,30,1,33,4,0,1.333333,0.666667,0.333333,0.333333,0.333333


# Training

## Random Forest

In [208]:
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)

In [209]:
rf.fit(x_train,y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [210]:
predict_rf=rf.predict(x_test)

### Evaluation

#### Accuracy 

In [211]:
from sklearn.metrics import accuracy_score

In [212]:
acc=accuracy_score(y_test,predict_rf)

In [213]:
acc

0.47688243064729197

## SVM

In [214]:
from sklearn.svm import SVC

In [215]:
svm = SVC(kernel='linear', C=1.0)
svm.fit(x_train, y_train)

SVC(kernel='linear')

In [216]:
pred_svm = svm.predict(x_test)

### Evaluation

#### Accuracy

In [217]:
acc_svm=accuracy_score(y_test,pred_svm)

In [218]:
acc_svm

0.4579480405107882

## XGBoost

In [219]:
from xgboost import XGBClassifier

In [220]:
model = XGBClassifier()
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [221]:
predXgb = model.predict(x_test)

### Evaluation

#### Accuracy

In [222]:
acc_xg=accuracy_score(y_test,predXgb)

In [223]:
acc_xg

0.4715984147952444

## Light GBM

In [224]:
import lightgbm as lgb

In [225]:
train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
params = {
    'objective': 'multiclass',
    'num_class': 3,  # Number of classes in the target variable
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}


In [226]:
num_round = 100
model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 7492, number of used features: 13
[LightGBM] [Info] Start training from score -1.405702
[LightGBM] [Info] Start training from score -0.976744
[LightGBM] [Info] Start training from score -0.972147


In [227]:
y_pred_prob = model.predict(x_test, num_iteration=model.best_iteration)
predGBM = np.argmax(y_pred_prob, axis=1)

### Evaluation

#### Accurracy

In [228]:
accuracy = accuracy_score(y_test, predGBM)

In [229]:
accuracy

0.4966974900924703

## CatBoost

In [230]:
from catboost import CatBoostClassifier


In [231]:
cat_features = None
y_train_cat = y_train.astype(str)
y_test_cat = y_test.astype(str)

In [232]:
model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass')

In [233]:
model.fit(x_train, y_train_cat, cat_features=cat_features, verbose=10)

0:	learn: 1.0883557	total: 3.98ms	remaining: 395ms
10:	learn: 1.0303084	total: 39.6ms	remaining: 320ms
20:	learn: 1.0076022	total: 75.8ms	remaining: 285ms
30:	learn: 0.9914137	total: 101ms	remaining: 226ms
40:	learn: 0.9796984	total: 128ms	remaining: 184ms
50:	learn: 0.9704790	total: 151ms	remaining: 145ms
60:	learn: 0.9615036	total: 175ms	remaining: 112ms
70:	learn: 0.9539300	total: 198ms	remaining: 81ms
80:	learn: 0.9452861	total: 227ms	remaining: 53.3ms
90:	learn: 0.9362761	total: 258ms	remaining: 25.5ms
99:	learn: 0.9271947	total: 280ms	remaining: 0us


In [234]:
predCat = model.predict(x_test)

### Evaluation

#### Accurracy

In [235]:
accuracy = accuracy_score(y_test_cat, predCat)

In [236]:
accuracy

0.5019815059445178